# **Title Name :  제주특산물가격예측**
------------------------------------------------------------
<p style="font-weight:bolder; font-size : 21px">
    Step : 3 앙상블
<p>
<p style="font-weight:bolder; font-size : 21px">
   RegDate : 2024.03.14



## 데이터셋 설명
---------------------------------

### train.csv
- item: 품목 코드
    - TG : 감귤
    - BC : 브로콜리
    - RD : 무
    - CR : 당근
    - CB : 양배추  
- corporation : 유통 법인 코드
    - 법인 A부터 F 존재 
- location : 지역 코드
    - J : 제주도 제주시
    - S : 제주도 서귀포시
- supply(kg) : 유통된 물량, kg 단위
- price(원/kg) : 유통된 품목들의 kg 마다의 가격, 원 단위

### international_trade.csv
- 관련 품목 수출입 정보
    - 중량 단위 kg
    - 금액 단위 천 달러

# 1. 환경설정
-------------------------------

In [27]:
#===============================================================================
# ▶ 모듈 불러오기
#===============================================================================

# 시스템 
import os 
import sys
import random
from tqdm import tqdm


# 데이터분석
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# 모델
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

# 평가지표생성
from sklearn.metrics import mean_squared_error, mean_absolute_error

# 훈련
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

# 최적화
from sklearn.model_selection import GridSearchCV

#전처리
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
import holidays



# 기타
import warnings
warnings.filterwarnings(action='ignore') 


In [14]:
#===============================================================================
# ▶ 작업환경
#===============================================================================
IS_GOOGLE = True if 'google.colab'                 in sys.modules   else False
IS_KAGGLE = True if 'KAGGLE_KERNEL_RUN_TYPE'       in os.environ    else False
IS_LOCAL  = True if  not (IS_GOOGLE or IS_KAGGLE)                   else False

In [15]:
#===============================================================================
# ▶ 시드설정
#===============================================================================

# 시드설정
SEED = 2024
def set_seed(SEED):
    random.seed(SEED)
    np.random.seed(SEED)
    # torch.manual_seed(SEED)
    # torch.cuda.manual_seed(SEED)
    os.environ['PYTHONHASHSEED'] = str(SEED)

# # deterministic 설정
# deterministic = True
# if deterministic:
#     torch.backends.cudnn.deterministic = True
#     torch.backends.cudnn.benchmark     = False

# set_seed(SEED)

In [16]:
#===============================================================================
# ▶ 데이터 패스설정
#===============================================================================

# 코랩용
if IS_GOOGLE:
    base_path = '/content/drive/MyDrive/프로젝트/제주특산물가격예측/data/'

# 캐글용
elif IS_KAGGLE :
    base_path = '/kaggle/input/__프로젝트명__/'

# 로컬용
elif IS_LOCAL :
    base_path = './data/'

# 트레인&테스트 경로
train_file = base_path+'train.csv'
test_file  = base_path+'test.csv'

# 기타 데이터 경로
international_trade  = base_path+'international_trade.csv'
sample_submission    = base_path+'sample_submission.csv'

# 결과 저장 경로
result_path = './res/'


# 2. 전처리
-------------------------------

In [17]:
#===============================================================================
# ▶ 데이터 불러오기
#===============================================================================

train_df = pd.read_csv(train_file)
test_df = pd.read_csv(test_file)
international_trade = pd.read_csv(international_trade)
submission = pd.read_csv(sample_submission)

In [18]:
#===============================================================================
# ▶ 테이블 확인
#===============================================================================
print('train')
print('='*150)
display(train_df)
print()

print('test_df')
print('='*150)
display(test_df)
print()

print('international_trade')
print('='*150)
display(international_trade)
print()

print('submission')
print('='*150)
print(submission)
print()

train


,ID,timestamp,item,corporation,location,supply(kg),price(원/kg)
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0
...,...,...,...,...,...,...,...
59392,RD_F_J_20230227,2023-02-27,RD,F,J,452440.0,468.0
59393,RD_F_J_20230228,2023-02-28,RD,F,J,421980.0,531.0
59394,RD_F_J_20230301,2023-03-01,RD,F,J,382980.0,574.0
59395,RD_F_J_20230302,2023-03-02,RD,F,J,477220.0,523.0



test_df


,ID,timestamp,item,corporation,location
0,TG_A_J_20230304,2023-03-04,TG,A,J
1,TG_A_J_20230305,2023-03-05,TG,A,J
2,TG_A_J_20230306,2023-03-06,TG,A,J
3,TG_A_J_20230307,2023-03-07,TG,A,J
4,TG_A_J_20230308,2023-03-08,TG,A,J
...,...,...,...,...,...
1087,RD_F_J_20230327,2023-03-27,RD,F,J
1088,RD_F_J_20230328,2023-03-28,RD,F,J
1089,RD_F_J_20230329,2023-03-29,RD,F,J
1090,RD_F_J_20230330,2023-03-30,RD,F,J



international_trade


,기간,품목명,수출 중량,수출 금액,수입 중량,수입 금액,무역수지
0,2019-01,토마토(신선한 것이나 냉장한 것으로 한정한다),356571,990,0,0,990
1,2019-01,양파,821330,222,4003206,1118,-896
2,2019-01,쪽파,60,1,93405,128,-127
3,2019-01,꽃양배추와 브로콜리(broccoli),160,1,638913,563,-562
4,2019-01,방울다다기 양배추,0,0,7580,38,-38
...,...,...,...,...,...,...,...
1269,2023-02,포포(papaw)[파파야(papaya)],0,0,23830,71,-71
1270,2023-02,사과,135165,351,0,0,351
1271,2023-02,배,2206012,5411,1,0,5411
1272,2023-02,신 체리[프루너스 체라서스(Prunus cerasus)],5,0,0,0,0



submission
                   ID  answer
0     TG_A_J_20230304       0
1     TG_A_J_20230305       0
2     TG_A_J_20230306       0
3     TG_A_J_20230307       0
4     TG_A_J_20230308       0
...               ...     ...
1087  RD_F_J_20230327       0
1088  RD_F_J_20230328       0
1089  RD_F_J_20230329       0
1090  RD_F_J_20230330       0
1091  RD_F_J_20230331       0

[1092 rows x 2 columns]



## 각 테이블 확인결과
------------------------

#### 01. train_df와 test_df 컬럼일치시키기
- train_df의 supply(kg)와 price(원/kg)를 제거하면 test_df와 같아짐

#### 02. 불필요한 컬럼 제거
- ID는 불필요함
- timestamp는 연,월,일,요일 등으로 파생피쳐 생성후 제거
- item,corporation,location 컬럼은 원핫인코딩 처리

#### train_X, train_y 분할
- X에 불필요한 컬럼제거(DataFrame)
- y에 타겟피쳐인 price만 추가(Series)

#### test_X
- 불필요한 컬럼제거

#### 훈련(학습) -> 예측(추론)
- train_X,train_Y로 학습
- test_X로 추론

In [19]:
#===============================================================================
# 01. 날짜 데이터 처리
#===============================================================================

# 한국 공휴일 라벨링

def make_holiday(x) :
    holiday = holidays.KR()
    if x in holiday:
        return 1
    else:
        return 0    

# timestamp 날짜형식으로 포맷
train_df['timestamp'] = pd.to_datetime(train_df['timestamp'], format='%Y-%m-%d')
test_df['timestamp'] = pd.to_datetime(test_df['timestamp'], format='%Y-%m-%d')

# 데이터셋에 날짜 추가
for df in [train_df, test_df]:
    df['year'] = df['timestamp'].dt.year
    df['month'] = df['timestamp'].dt.month
    df['day'] = df['timestamp'].dt.day
    df['weekday'] = df['timestamp'].dt.weekday
    df['day_of_year'] = df['timestamp'].dt.day_of_year
    df['week_of_year'] = df['timestamp'].dt.isocalendar().week
    df["holiday"] = df["timestamp"].map(lambda x : make_holiday(x))
    
    # 컬럼 이름변경
    df.rename(columns={'supply(kg)':'supply', 'price(원/kg)':'price'},inplace=True)

    
train_df
# test_df

,ID,timestamp,item,corporation,location,supply,price,year,month,day,weekday,day_of_year,week_of_year,holiday
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,1,1,1,1
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019,1,2,2,2,1,0
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019,1,3,3,3,1,0
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,2019,1,4,4,4,1,0
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,2019,1,5,5,5,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59392,RD_F_J_20230227,2023-02-27,RD,F,J,452440.0,468.0,2023,2,27,0,58,9,0
59393,RD_F_J_20230228,2023-02-28,RD,F,J,421980.0,531.0,2023,2,28,1,59,9,0
59394,RD_F_J_20230301,2023-03-01,RD,F,J,382980.0,574.0,2023,3,1,2,60,9,1
59395,RD_F_J_20230302,2023-03-02,RD,F,J,477220.0,523.0,2023,3,2,3,61,9,0


In [20]:
#===============================================================================
# 02. supply mean 파생피쳐 추가
#===============================================================================
supply_mean = train_df.groupby(['item', 'corporation', 'week_of_year'])['supply'].mean().reset_index()
train_df = pd.merge(train_df, supply_mean, left_on=['item', 'corporation', 'week_of_year'], right_on=['item', 'corporation', 'week_of_year'], how='left').dropna()
test_df = pd.merge(test_df, supply_mean, left_on=['item', 'corporation', 'week_of_year'], right_on=['item', 'corporation', 'week_of_year'], how='left').dropna()
train_df.rename(columns={'supply_x': 'supply', 'supply_y': 'supply_week_of_year_mean', 'price(원/kg)': 'price'}, inplace=True)
test_df.rename(columns={'supply': 'supply_week_of_year_mean'}, inplace=True)

train_df

,ID,timestamp,item,corporation,location,supply,price,year,month,day,weekday,day_of_year,week_of_year,holiday,supply_week_of_year_mean
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,1,1,1,1,52808.920588
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019,1,2,2,2,1,0,52808.920588
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019,1,3,3,3,1,0,52808.920588
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,2019,1,4,4,4,1,0,52808.920588
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,2019,1,5,5,5,1,0,52808.920588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59392,RD_F_J_20230227,2023-02-27,RD,F,J,452440.0,468.0,2023,2,27,0,58,9,0,376770.303030
59393,RD_F_J_20230228,2023-02-28,RD,F,J,421980.0,531.0,2023,2,28,1,59,9,0,376770.303030
59394,RD_F_J_20230301,2023-03-01,RD,F,J,382980.0,574.0,2023,3,1,2,60,9,1,376770.303030
59395,RD_F_J_20230302,2023-03-02,RD,F,J,477220.0,523.0,2023,3,2,3,61,9,0,376770.303030


In [21]:
#===============================================================================
# 03. 피쳐추가제거 
#===============================================================================
# 불필요한 피쳐 제거
train_x = train_df.drop(columns=['ID', 'timestamp', 'supply', 'price'])
train_y = train_df['price'] 
test_x = test_df.drop(columns=['ID', 'timestamp'])


In [22]:
#질적 변수들을 수치화합니다
qual_col = ['item', 'corporation', 'location']

for i in qual_col:
    le = LabelEncoder()
    train_x[i]=le.fit_transform(train_x[i])
    test_x[i]=le.transform(test_x[i]) #test 데이터에 대해서 fit하는 것은 data leakage에 해당합니다

print('Done.')

Done.


# 3. 하이퍼파라미터 최적화
------------------

In [32]:
#===============================================================================
# 하이퍼파라미터 세팅 
#===============================================================================
LEARNIN_GRATE = 1e-3
VERBOSE = 1000
N = 1e+4
KF = 5

In [ ]:
# #Cat Boost 하이퍼파라미터 탐색
# grid_search = GridSearchCV(model_pipeline, param_grid, cv=5, scoring='neg_mean_squared_error', verbose=1)
# grid_search.fit(X_train, y_train)

# #최적의 하이퍼파라미터
# print("Best parameters:", grid_search.best_params_)

# #최적의 하이퍼파라미터를 사용하여 모델 재구성
# model_pipeline.set_params(**grid_search.best_params_)


# 4. 훈련및 검증
------------------

In [33]:
# 모델선정
model = CatBoostRegressor(n_estimators = N, verbose = VERBOSE, learning_rate = LEARNIN_GRATE)

# K-Fold 설정
kf = KFold(n_splits=KF, shuffle=True, random_state=SEED)

# 각 fold의 모델로부터의 예측을 저장할 리스트와 MAE 점수 리스트
ensemble_predictions = []
scores = []

# 데이터분할
for train_idx, val_idx in tqdm(kf.split(train_x), total=KF, desc="Processing folds"):
    X_train, X_val = train_x.iloc[train_idx], train_x.iloc[val_idx]
    y_train, y_val = train_y[train_idx], train_y[val_idx]
    
    # 두 모델 모두 학습
    model.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=10, use_best_model=True)
    
    # 각 모델로부터 Validation set에 대한 예측을 평균내어 앙상블 예측 생성
    val_pred = model.predict(X_val)
    
    # Validation set에 대한 대회 평가 산식 계산 후 저장
    scores.append(mean_squared_error(y_val, val_pred)**0.5)
    print(mean_squared_error(y_val, val_pred)**0.5)
    
    # test 데이터셋에 대한 예측 수행 후 저장
    lgbm_pred = model.predict(test_x)
    lgbm_pred = np.where(lgbm_pred < 0, 0, lgbm_pred)
    
    ensemble_predictions.append(lgbm_pred)

# K-fold 모든 예측의 평균을 계산하여 fold별 모델들의 앙상블 예측 생성
final_predictions = np.mean(ensemble_predictions, axis=0)

# 각 fold에서의 Validation Metric Score와 전체 평균 Validation Metric Score출력
print("Validation : MAE scores for each fold:", scores)
print("Validation : MAE:", np.mean(scores))    

Processing folds:   0%|          | 0/2 [00:00<?, ?it/s]

0:	learn: 2033.7521904	test: 2023.4076514	best: 2023.4076514 (0)	total: 3.75ms	remaining: 37.5s
1000:	learn: 1297.9462549	test: 1282.0970784	best: 1282.0970784 (1000)	total: 2.91s	remaining: 26.2s
2000:	learn: 1119.0556912	test: 1099.2722406	best: 1099.2722406 (2000)	total: 5.81s	remaining: 23.2s
3000:	learn: 1061.3440523	test: 1041.0942053	best: 1041.0942053 (3000)	total: 8.76s	remaining: 20.4s
4000:	learn: 1029.9091588	test: 1010.7450552	best: 1010.7450552 (4000)	total: 11.8s	remaining: 17.6s
5000:	learn: 1007.6652894	test: 990.1424048	best: 990.1424048 (5000)	total: 14.7s	remaining: 14.7s
6000:	learn: 991.0896445	test: 975.2689686	best: 975.2689686 (6000)	total: 17.6s	remaining: 11.7s
7000:	learn: 978.3196123	test: 964.3627144	best: 964.3627144 (7000)	total: 20.4s	remaining: 8.75s
8000:	learn: 967.7064344	test: 955.5206621	best: 955.5206621 (8000)	total: 23.3s	remaining: 5.82s
9000:	learn: 958.8430440	test: 948.4208235	best: 948.4208235 (9000)	total: 26.2s	remaining: 2.91s
9999:	lea

Processing folds:  50%|█████     | 1/2 [00:30<00:30, 30.28s/it]

0:	learn: 2023.3117423	test: 2033.8335959	best: 2033.8335959 (0)	total: 4.05ms	remaining: 40.5s
1000:	learn: 1276.1952804	test: 1295.1211680	best: 1295.1211680 (1000)	total: 2.93s	remaining: 26.3s
2000:	learn: 1093.0898756	test: 1120.1973729	best: 1120.1973729 (2000)	total: 5.89s	remaining: 23.6s
3000:	learn: 1033.9136878	test: 1064.9338353	best: 1064.9338353 (3000)	total: 8.79s	remaining: 20.5s
4000:	learn: 1002.8586353	test: 1036.4279729	best: 1036.4279729 (4000)	total: 11.7s	remaining: 17.6s
5000:	learn: 981.5955319	test: 1017.5805685	best: 1017.5805685 (5000)	total: 14.7s	remaining: 14.7s
6000:	learn: 965.4326692	test: 1003.7236831	best: 1003.7236831 (6000)	total: 17.6s	remaining: 11.7s
7000:	learn: 952.5857454	test: 993.0137344	best: 993.0137344 (7000)	total: 20.5s	remaining: 8.8s
8000:	learn: 941.7654336	test: 984.3752174	best: 984.3752174 (8000)	total: 23.6s	remaining: 5.89s
9000:	learn: 932.1782523	test: 976.7747663	best: 976.7747663 (9000)	total: 26.5s	remaining: 2.94s


Processing folds: 100%|██████████| 2/2 [01:00<00:00, 30.43s/it]

9999:	learn: 923.5329083	test: 970.1334151	best: 970.1334151 (9999)	total: 29.4s	remaining: 0us

bestTest = 970.1334151
bestIteration = 9999

970.1334151072409
Validation : MAE scores for each fold: [942.0952237822908, 970.1334151072409]
Validation : MAE: 956.1143194447659


In [36]:
test_df['price'] = final_predictions
test_df.loc[test_df.weekday == 6, 'price'] = 0


In [37]:
len(final_predictions)

1092

# 5. 제출(inference)
-------------------------------

In [38]:
submission['answer'] = test_df['price']
submission

,ID,answer
0,TG_A_J_20230304,2828.822976
1,TG_A_J_20230305,0.000000
2,TG_A_J_20230306,3435.621920
3,TG_A_J_20230307,3444.703852
4,TG_A_J_20230308,3440.396225
...,...,...
1087,RD_F_J_20230327,628.025464
1088,RD_F_J_20230328,626.124804
1089,RD_F_J_20230329,620.388813
1090,RD_F_J_20230330,620.231373


In [39]:
submission.to_csv(result_path+'_ensemble_'+'baseline_submission.csv', index=False)